![title](https://user-images.githubusercontent.com/16294340/44064358-9f6f5a12-9f32-11e8-9516-fee9aac92884.png)

# Wrangling *Income by Race* data using R
## BDPA Confernce 2018 

In the Notebook we will use three R packages to cleans and reshape the income data into a form that is conducive for data analytics. The three packages we will use are readxl, dplyr, and tidyr. The readxl package will be used to load data from the CensusIncomeData Excel workbook into a R dataframe. The dplyr package from tidyverse will be used to wrangle and shape the data. And the tidyr package will be used to unpivot the data and put it in a "tidy" format.

In [1]:
library(readxl)
library(tidyverse)

-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 2.2.1     v purrr   0.2.4
v tibble  1.4.2     v dplyr   0.7.5
v tidyr   0.8.1     v stringr 1.3.1
v readr   1.1.1     v forcats 0.3.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


We first need to set the working directory to the path of the folder where the CensusIncomeData.xlsx file is located. We then use the read_excel function from the readxl package to read in the data from the workbook into a R dataframe. Note that we explicitly reference the sheet name and the number of rows we want to skip.

In [2]:
#setwd("<path where the CensusIncomeData.xlsx file is located")
setwd("C:/Users/rwade/OneDrive - Diesel Analytics/Talks/BDPA/Code/TestCode/")
unformatted_data <- 
    read_excel(
        "./Data/CensusIncomeData.xlsx", 
        sheet = "a1", 
        skip = 3, 
        col_names = FALSE
    )

The character vector below will be used later on in the notebook to identify the races

In [3]:
races <- c("ALL RACES", "WHITE ALONE", "BLACK ALONE", "ASIAN ALONE", "HISPANIC")

In our first transformation step we will create a new dataframe and will assign to it the content of the unformatted_data dataframe

In [4]:
transformation_step_1 <- unformatted_data

In our next transformation step we will create a column that will be used to identify the race. The race is only identified at the begin of the section for each race. In the row that it is identified the other columns are NA's. Using that information create a calculated column that looks at all of the columns in 2-16 and if they are NA's we know that we are in a row that is identifying the race and we can use the value from X__1 as the race. We can ignore all other rows.

In [6]:
transformation_step_2 <-
    mutate(transformation_step_1,
        Race =
            ifelse(
                is.na(X__2) & is.na(X__3) & is.na(X__4) & is.na(X__5) & is.na(X__6) &
                is.na(X__7) & is.na(X__8) & is.na(X__9) & is.na(X__10) & is.na(X__11) &
                is.na(X__12) & is.na(X__13) & is.na(X__14) & is.na(X__15) & is.na(X__16),
                X__1,
                NA
            )
    )

We now have a situation where the Race field is only populated in instances where columns 2:16 had NA's. We want to fill the race information down until we get to a row that has a race in it. We can easily do that using the fill function to accomplish that.

In [7]:
transformation_step_3 <- 
    fill(
        transformation_step_2, 
        Race
    )

At this point we have the Race column field but we have situations where the same race is labeled differently. For instance in our first example you see that "WHITE ALONE" is named differently. In the block of code below we give each race a consistent name.

In [8]:
transformation_step_4 <-
    mutate(
        transformation_step_3,
        Race =
            ifelse(Race == "WHITE ALONE 22" | Race == "WHITE 23", "WHITE ALONE",
            ifelse(Race == "BLACK ALONE 24" | Race == "BLACK 23", "BLACK ALONE",
            ifelse(Race == "ASIAN ALONE 25" | Race == "ASIAN AND PACIFIC ISLANDER 23", "ASIAN ALONE",
            ifelse(Race == "HISPANIC (ANY RACE) 26", "HISPANIC",
            Race))))
    )

The data in rows 1:3 are header information that we don't need. We tell R that we want rows 4 to the last row using via the slice function. We do so by passing "4:n()" as the second argument to the slice function. The 4 tells the slice function where we want to start and the n() function count the number of rows in the dataframe thus dynamically giving us the last row. 

In [9]:
transformation_step_5 <- 
    slice(transformation_step_4, 4:n())

We want to identify all of the rows that starts with a 4 digit string that represents a year. The rows that does not we want to exclude. Through inspecting the data we noticed that there were some years we multiple rows were provided for the same year and same race. We made the decision that we wanted to keep just the first one. We also noticed that the second one aloways had a superscripted "2". It is relatively easy to identify rows that starts with 4 numeric digits but to exclude ones that starts with 4 digists but is not followed by a space and a 2 at the end of the line is pretty challenging. Fortunately that is straight forward with regular expressions. We use a regular expression below to extract the year from rows that starts with 4 numeric digits but is not followed by a space and 2 at the end of the line.

In [10]:
transformation_step_6 <-
    mutate(
        transformation_step_5, X__1 = str_extract(X__1, "^\\d{4}(?!\\s[2]$)")
    )

We rename our columns

In [11]:
transformation_step_7 <-
    rename(
        transformation_step_6,
        Year = X__1, Population = X__2, Total = X__3, `< 15K` = X__4,
        `[15 - 25K)` = X__5, `[25 - 35K)` = X__6, `[35 - 50K)` = X__7,
        `[50 - 75K)` = X__8, `[75 - 100K)` = X__9, `[100 - 150K)` = X__10,
        `[150 - 200K)` = X__11, `>= 200K` = X__12, `Median Income` = X__13,
        `Median Income SE` = X__14, `Mean Income` = X__15, `Mean Income SE` = X__16
    )

We filter out all rows that either has NA in the Year field and is one of the races in the races character vector in the Race field.

In [12]:
transformation_step_8 <-
    filter(
        transformation_step_7, !is.na(Year) & Race %in% races
    )

All of the columns in our dataframe are in the character data type but all but the Race column should be in a numeric data type. We can easiy change the data type of each field to a numeric data type using the mutate_if function as represented in the code snippet below.

In [13]:
transformation_step_9 <-
    mutate_if(
        transformation_step_8, !(names(transformation_step_8) %in% c("Race")), as.numeric
    )

Warning message in evalq(as.numeric(Population), <environment>):
"NAs introduced by coercion"Warning message in evalq(as.numeric(`Mean Income`), <environment>):
"NAs introduced by coercion"Warning message in evalq(as.numeric(`Mean Income SE`), <environment>):
"NAs introduced by coercion"

We create a dataframe that represents income data and we subset the following fields for the dataframe.

In [14]:
income_data <- 
    select(
        transformation_step_9, 
        Race, 
        Year, 
        Population, 
        `Mean Income`, 
        `Median Income`
    )


We create a named numeric vector with a length of 9 that will be used later on.

In [15]:
ranges <- c("< 15K", "[15 - 25K)", "[25 - 35K)", "[35 - 50K)", "[50 - 75K)", "[75 - 100K)", "[100 - 150K)", "[150 - 200K)", ">= 200K")
range_id <- seq(1:9)
names(range_id) <- ranges

We create a named numeric vector with a length of 5 that will be used later on

In [16]:
race <- c("ALL RACES", "WHITE ALONE", "BLACK ALONE", "ASIAN ALONE", "HISPANIC")
race_id <- c(5, 3, 1, 4, 2)
names(race_id) <- race

Write some stuff here

In [17]:
tidy_income_distribution <-
    transformation_step_9 %>%
    select(
        Race, Year, `< 15K`, `[15 - 25K)`, `[25 - 35K)`, `[35 - 50K)`, `[50 - 75K)`, 
        `[75 - 100K)`, `[100 - 150K)`, `[150 - 200K)`, `>= 200K`
    ) %>%
    gather(
        key = "Range", 
        value = "Percent", 
        - Race, - Year
    ) %>%
    mutate(
        RangeID = range_id[Range],
        RaceID = race_id[Race]
    ) %>%
    select(
        Year, RangeID, RaceID, Race, Range, Percent
    ) %>%
    arrange(
        RaceID, Year, RangeID
    )

Visualize the income dataframe

In [18]:
head(income_data)

Race,Year,Population,Mean Income,Median Income
ALL RACES,2016,126224,83143,59039
ALL RACES,2015,125819,80265,57230
ALL RACES,2014,124587,76783,54398
ALL RACES,2013,123931,77480,55214
ALL RACES,2012,122459,74507,53331
ALL RACES,2011,121084,74336,53401


Visualize the tidy_income_distribution dataframe

In [19]:
head(tidy_income_distribution)

Year,RangeID,RaceID,Race,Range,Percent
1967,1,1,BLACK ALONE,< 15K,28.7
1967,2,1,BLACK ALONE,[15 - 25K),17.8
1967,3,1,BLACK ALONE,[25 - 35K),15.9
1967,4,1,BLACK ALONE,[35 - 50K),16.1
1967,5,1,BLACK ALONE,[50 - 75K),13.8
1967,6,1,BLACK ALONE,[75 - 100K),4.6
